In [1]:

import torch
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

In [2]:
# Define o computador utilizado como cuda (gpu) se existir ou cpu caso contrário
print(torch.cuda.is_available())
dev = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
torch.set_default_dtype(torch.float64)
torch.manual_seed(42)
np.random.seed(42)

True


In [22]:
import sys
import os
sys.path.insert(1, '..')
sys.path.insert(1, '../..')

import models
from utils import measures,metrics